In [2]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from get_12ECG_features import get_12ECG_features
from driver import load_challenge_data

import sklearn
from sklearn.metrics import mutual_info_score

In [3]:
# Define directories here
data_dir = "/share/workhorse3/vsanil/physionet/Training_WFDB"
data_json_path = "/share/workhorse3/vsanil/physionet/"

In [4]:
# Define program variables here
data_dict = {}

In [7]:
# Extracts features and stores as a dictionary
def extract_all_data(path_name, data_dict):
    # Walk through directory
    for root, dirs, files in os.walk(path_name):
        for idx, filename in enumerate(files):
            if filename.endswith('.mat'):
                print(idx)
                data, header = load_challenge_data(os.path.join(root, filename))
                data_dict[filename.replace('.mat','')] = [data, header]
                
    return data_dict

# Save data in pickle format
def save_pickle(path, data_dict, filename):
    with open(os.path.join(path, '{}.p'.format(filename)), 'wb') as fp:
        pickle.dump(data_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
# Get label dict
def get_labels(data_dict):
    label_dict = {}
    for filename, feature_list in data_dict.items():
        label = feature_list[1][-4].replace("#Dx: ","").replace("\n","").split(',')
        for l in label:
            if l not in label_dict:
                label_dict[l] = 0
        
    return label_dict

In [ ]:
# Check if model dictionary is already present
if os.path.exists(os.path.join(data_json_path, 'data.p')):
    with open(os.path.join(data_json_path, 'data.p'), 'rb') as fp:
        data_dict = pickle.load(fp)
else:
    data_dict = extract_all_data(data_dir, data_dict)
    print("Data extracted")
    #save_pickle(data_json_path, data_dict, "data")
    
# Generate label dictionary
#label_dict = get_labels(data_dict)
#sex_dict = {"Male":0, "Female":1}

9
10
13
16
19
22
23
26
28
29
35
36
37
40
42
43
46
47
49
50
51
53
54
55
56
58
61
63
64
65
70
73
74
76
79
83
87
89
92
93
95
96
98
99
107
108
113
114
116
120
121
127
129
131
133
135
136
137
139
144
146
149
150
153
154
155
158
159
161
163
165
167
168
169
172
174
176
177
182
183
184
188
189
192
193
196
203
204
205
209
211
212
214
215
216
217
222
226
227
228
229
230
232
236
238
239
245
248
249
253
254
256
257
260
261
267
270
274
278
279
281
286
287
288
289
292
294
295
297
298
301
304
305
309
312
313
314
316
320
323
326
327
329
334
337
338
341
344
347
348
352
353
354
356
357
362
365
366
368
369
371
374
376
377
379
380
381
385
386
388
389
390
392
393
394
396
399
401
402
403
404
405
407
408
409
410
411
412
414
417
420
422
423
427
428
429
430
432
433
436
438
439
441
443
446
447
449
450
452
453
455
456
458
461
463
464
467
469
474
476
478
479
481
482
484
485
488
489
490
491
493
494
496
497
498
499
501
502
503
504
505
507
508
512
513
514
515
516
518
519
521
522
523
525
527
528
529
530
531
533
534
5

3505
3508
3509
3510
3511
3514
3516
3519
3520
3521
3522
3523
3524
3526
3527
3530
3533
3535
3536
3537
3538
3540
3542
3544
3545
3546
3547
3551
3554
3555
3560
3563
3564
3566
3569
3570
3571
3572
3573
3577
3579
3581
3582
3584
3585
3586
3588
3589
3590
3591
3595
3596
3597
3599
3602
3603
3604
3605
3608
3609
3610
3611
3612
3613
3614
3617
3618
3621
3624
3626
3627
3629
3632
3634
3636
3637
3638
3640
3642
3644
3646
3648
3652
3654
3655
3657
3661
3662
3663
3664
3665
3666
3669
3671
3672
3673
3677
3678
3679
3681
3682
3683
3684
3685
3686
3687
3689
3691
3693
3694
3696
3698
3700
3701
3703
3708
3709
3715
3717
3718
3721
3724
3725
3726
3728
3729
3732
3733
3734
3737
3739
3740
3741
3742
3748
3750
3751
3756
3758
3759
3763
3764
3765
3768
3769
3771
3772
3774
3775
3782
3784
3788
3790
3793
3794
3796
3798
3799
3802
3803
3805
3807
3814
3816
3818
3823
3824
3826
3827
3831
3832
3836
3839
3840
3842
3847
3848
3852
3853
3856
3858
3859
3863
3864
3866
3868
3870
3871
3877
3882
3886
3890
3895
3896
3897
3901
3903
3905
3906
3910


6849
6851
6853
6854
6857
6861
6862
6863
6865
6866
6869
6873
6875
6878
6881
6882
6883
6884
6885
6886
6889
6897
6898
6903
6904
6907
6910
6911
6912
6916
6917
6919
6920
6925
6928
6929
6931
6936
6938
6939
6944
6945
6953
6955
6959
6960
6961
6963
6966
6968
6971
6972
6973
6975
6976
6978
6980
6982
6984
6986
6987
6992
6993
6994
6996
6999
7002
7005
7008
7009
7012
7015
7019
7023
7024
7027
7028
7035
7040
7041
7042
7047
7049
7050
7051
7054
7056
7060
7063
7064
7065
7066
7067
7071
7072
7074
7075
7078
7079
7082
7083
7084
7085
7089
7090
7091
7092
7093
7097
7099
7100
7102
7105
7106
7107
7108
7109
7112
7115
7116
7122
7123
7124
7125
7126
7128
7130
7131
7132
7133
7134
7135
7137
7140
7141
7143
7146
7147
7151
7152
7155
7156
7158
7161
7163
7164
7165
7166
7168
7170
7171
7172
7173
7174
7176
7178
7179
7180
7181
7183
7186
7187
7191
7193
7194
7195
7197
7198
7200
7201
7202
7204
7205
7206
7209
7210
7211
7212
7213
7216
7218
7220
7223
7226
7227
7228
7230
7231
7233
7234
7237
7238
7242
7243
7245
7246
7248
7249
7252
7255


10108
10110
10111
10112
10119
10120
10122
10123
10126
10129
10131
10132
10133
10136
10137
10138
10140
10142
10146
10148
10149
10153
10155
10157
10158
10159
10162
10163
10164
10167
10168
10169
10171
10172
10174
10175
10176
10178
10179
10180
10181
10183
10184
10185
10186
10189
10190
10191
10196
10197
10198
10202
10206
10207
10209
10210
10211
10213
10215
10216
10218
10220
10223
10224
10225
10226
10228
10229
10231
10232
10233
10236
10237
10238
10239
10242
10243
10244
10245
10248
10249
10253
10254
10256
10257
10258
10259
10260
10262
10264
10265
10266
10267
10270
10274
10275
10277
10278
10279
10282
10283
10286
10288
10289
10290
10291
10292
10296
10297
10299
10302
10303
10304
10306
10307
10308
10309
10310
10311
10313
10314
10320
10323
10324
10326
10327
10328
10329
10332
10335
10336
10337
10338
10339
10342
10344
10347
10348
10349
10350
10351
10354
10355
10356
10358
10360
10363
10364
10365
10366
10367
10368
10372
10373
10374
10376
10377
10380
10382
10383
10385
10386
10387
10388
10392
10393
1039

12863
12864
12867
12871
12872
12873
12875
12878
12879
12882
12883
12885
12888
12889
12891
12892
12893
12895
12896
12901
12902
12903
12904
12905
12906
12909
12912
12914
12916
12917
12918
12919
12920
12922
12923
12925
12926
12927
12933
12935
12939
12940
12942
12944
12945
12947
12950
12952
12953
12954
12955
12956
12957
12959
12960
12961
12963
12965
12966
12967
12972
12974
12975
12976
12977
12979
12980
12982
12983
12984
12985
12986
12989
12991
12992
12993
12994
12996
12998
12999
13000
13001
13002
13007
13009
13010
13011
13012
13014
13017
13019
13021
13023
13024
13026
13028
13029
13031
13039
13040
13041
13045
13046
13049
13050
13055
13056
13057
13059
13061
13063
13064
13066
13071
13072
13073
13074
13076
13078
13079
13081
13082
13083
13085
13086
13089
13090
13091
13094
13095
13097
13100
13101
13102
13103
13104
13105
13107
13112
13113
13115
13117
13118
13120
13122
13123
13126
13127
13128
13129
13133
13136
13137
13139
13142
13148
13149
13150
13152
13156
13158
13159
13164
13165
13168
13170
1317

...

29383
29384
29388
29389
29391
29392
29393
29394
29397
29399
29400
29401
29404
29407
29408
29410
29413
29416
29418
29419
29421
29424
29427
29428
29433
29436
29437
29439
29443
29444
29445
29447
29449
29450
29452
29455
29458
29461
29464
29465
29466
29469
29473
29474
29477
29478
29479
29481
29484
29487
29488
29489
29491
29492
29493
29497
29498
29500
29503
29504
29507
29510
29514
29515
29519
29521
29524
29526
29527
29529
29530
29531
29541
29542
29546
29550
29551
29552
29554
29556
29557
29559
29560
29561
29567
29569
29572
29574
29577
29578
29582
29583
29584
29588
29590
29591
29592
29598
29600
29604
29605
29607
29608
29610
29613
29620
29622
29625
29626
29628
29632
29633
29638
29639
29641
29642
29643
29644
29646
29649
29651
29652
29653
29654
29657
29658
29660
29662
29663
29664
29666
29667
29670
29671
29674
29676
29679
29680
29682
29683
29684
29687
29689
29691
29692
29695
29696
29698
29702
29703
29705
29706
29707
29708
29709
29712
29714
29715
29717
29718
29720
29722
29723
29724
29725
29726
2972

32162
32166
32168
32169
32171
32176
32179
32182
32185
32188
32189
32191
32195
32196
32200
32201
32202
32204
32207
32208
32210
32214
32215
32216
32217
32220
32222
32223
32226
32228
32229
32231
32232
32235
32237
32239
32240
32247
32249
32252
32253
32258
32259
32262
32266
32268
32271
32277
32278
32282
32283
32285
32286
32288
32290
32293
32294
32297
32299
32300
32301
32302
32303
32304
32307
32310
32313
32314
32317
32320
32321
32322
32325
32326
32327
32328
32329
32330
32334
32335
32336
32338
32339
32340
32342
32343
32344
32345
32348
32352
32353
32354
32355
32356
32357
32359
32360
32362
32363
32365
32368
32369
32373
32374
32375
32378
32380
32382
32383
32386
32388
32390
32392
32396
32399
32400
32402
32403
32406
32408
32409
32410
32412
32413
32415
32418
32419
32420
32421
32423
32425
32426
32428
32432
32433
32434
32435
32436
32438
32439
32440
32441
32442
32443
32448
32450
32454
32455
32456
32457
32459
32460
32464
32465
32468
32470
32471
32472
32473
32474
32475
32478
32479
32482
32483
32484
3248

34907
34909
34910
34912
34914
34917
34918
34920
34921
34922
34924
34926
34927
34930
34934
34937
34938
34942
34943
34944
34945
34947
34948
34953
34957
34958
34961
34962
34963
34964
34966
34970
34971
34974
34976
34977
34979
34981
34986
34987
34988
34991
34994
34995
34997
35003
35006
35008
35009
35011
35013
35014
35018
35020
35023
35024
35025
35026
35027
35028
35031
35032
35036
35037
35038
35039
35040
35042
35043
35044
35048
35052
35053
35055
35056
35058
35061
35062
35063
35064
35068
35070
35073
35074
35076
35079
35080
35081
35082
35083
35086
35087
35088
35090
35092
35093
35099
35100
35101
35105
35106
35110
35111
35112
35113
35114
35115
35116
35117
35119
35122
35125
35126
35127
35128
35134
35135
35136
35137
35138
35139
35140
35141
35142
35145
35147
35148
35150
35152
35153
35157
35159
35161
35162
35164
35165
35167
35168
35171
35172
35174
35176
35177
35180
35182
35183
35185
35187
35189
35190
35192
35196
35198
35199
35200
35201
35202
35204
35206
35209
35210
35212
35213
35214
35215
35217
3521

37701
37704
37707
37709
37710
37712
37715
37716
37718
37723
37724
37726
37727
37730
37731
37735
37737
37740
37741
37744
37747
37754
37758
37759
37761
37762
37763
37765
37769
37771
37772
37776
37777
37779
37780
37781
37785
37786
37790
37791
37793
37795
37796
37798
37804
37805
37807
37809
37810
37811
37814
37815
37817
37824
37825
37829
37836
37837
37838
37841
37845
37846
37847
37852
37853
37855
37858
37860
37861
37862
37863
37866
37868
37871
37872
37878
37879
37880
37881
37883
37885
37886
37887
37892
37896
37897
37898
37900
37901
37903
37905
37906
37908
37914
37915
37918
37919
37921
37922
37925
37926
37927
37932
37933
37934
37939
37940
37942
37945
37947
37949
37951
37952
37956
37957
37958
37960
37963
37967
37972
37974
37977
37979
37980
37983
37985
37986
37987
37990
37994
37995
37997
38000
38003
38004
38006
38009
38010
38016
38018
38019
38021
38027
38028
38033
38035
38036
38037
38040
38043
38044
38046
38047
38049
38050
38051
38058
38059
38060
38061
38066
38067
38069
38070
38071
38072
3807

40509
40510
40513
40517
40518
40519
40522
40524
40526
40527
40528
40530
40535
40537
40539
40542
40543
40545
40546
40550
40551
40552
40556
40557
40567
40569
40570
40572
40578
40579
40581
40583
40586
40588
40590
40592
40598
40599
40600
40601
40603
40605
40606
40607
40608
40609
40612
40617
40618
40622
40624
40628
40630
40632
40633
40636
40639
40641
40643
40645
40646
40648
40650
40654
40661
40662
40663
40667
40668
40670
40671
40674
40677
40678
40681
40684
40689
40691
40694
40695
40704
40707
40708
40709
40710
40712
40713
40716
40717
40722
40727
40729
40730
40733
40734
40735
40739
40741
40742
40743
40744
40747
40750
40751
40752
40753
40756
40757
40758
40763
40764
40765
40766
40767
40769
40770
40772
40774
40775
40779
40780
40781
40783
40784
40785
40786
40788
40789
40790
40791
40793
40797
40798
40800
40801
40804
40805
40806
40808
40809
40812
40813
40815
40817
40822
40823
40825
40826
40828
40831
40832
40835
40836
40838
40840
40841
40842
40843
40845
40847
40848
40849
40852
40853
40855
40856
4085

In [1]:
data_dict

NameError: name 'data_dict' is not defined

In [14]:
obs_dict = {'Age':[], 'Sex':[], 'Hashed Label':[], 'Label':[], 'Feature':[], 'Filename':[]}
feature_hash = {}
count = 0

for filename, feature_list in data_dict.items():
    feat_lead_12 = feature_list[0]
    header_data = feature_list[1]
    ind_obs_l_dict = {}
    
    # Get metadata 
    label = header_data[-4].replace("#Dx: ","").replace("\n","").split(',')
    age = header_data[-6].replace("#Age: ","").replace("\n","").split(',')[0]
    sex = sex_dict[header_data[-5].replace("#Sex: ","").replace("\n","").split(',')[0]]
    
    if age=="NaN":
        continue
    
    # Generate labels
    for k,v in label_dict.items():
        if k in label:
            ind_obs_l_dict[k] = 1
        else:
            ind_obs_l_dict[k] = 0
    feat_label = list(ind_obs_l_dict.values())
    if tuple(feat_label) not in feature_hash:
        feature_hash[tuple(feat_label)] = count
        count += 1
    
    # Construct a observation dictionary
    obs_dict['Age'].append(int(age))
    obs_dict['Sex'].append(sex)
    obs_dict['Hashed Label'].append(feature_hash[tuple(feat_label)])
    obs_dict['Label'].append(feat_label)
    obs_dict['Feature'].append(feat_lead_12)
    obs_dict['Filename'].append(filename)
    
    # Display header
    #for h in header_data:
    #    print(h[:-1])
    
df = pd.DataFrame(obs_dict, columns = list(obs_dict.keys()))

In [15]:
df

,Age,Sex,Hashed Label,Label,Feature,Filename
0,28,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[[73.0, 122.0, 121.0, 135.0, 126.0, 111.0, 108...",A4086
1,17,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]","[[-36.0, -51.0, -47.0, -46.0, -38.0, -44.0, -4...",A4721
2,90,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]","[[6.0, 5.0, 9.0, 8.0, 4.0, 8.0, 10.0, 10.0, 8....",A0198
3,63,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[[-77.0, -75.0, -72.0, -65.0, -61.0, -60.0, -5...",A5161
4,51,0,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[[102.0, 153.0, 145.0, 155.0, 158.0, 158.0, 15...",A6074
...,...,...,...,...,...,...
6867,76,1,10,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[[-36.0, -52.0, -47.0, -49.0, -48.0, -48.0, -4...",A3692
6868,27,1,5,"[0, 0, 0, 0, 0, 1, 0, 0, 0]","[[14.0, 12.0, 13.0, 12.0, 11.0, 10.0, 10.0, 8....",A0020
6869,82,0,8,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[[15.0, 8.0, 8.0, 17.0, 6.0, 2.0, -25.0, -53.0...",A0787
6870,39,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[[34.0, 33.0, 33.0, 27.0, 17.0, 4.0, -12.0, -2...",A1660


In [16]:
df.corr()

,Age,Sex,Hashed Label
Age,1.000000,-0.130352,0.085924
Sex,-0.130352,1.000000,-0.051748
Hashed Label,0.085924,-0.051748,1.000000


In [29]:
save_pickle(data_json_path, obs_dict,"observations")